In [145]:
from eth_rpc import *

# create a new wallet
wallet = PrivateKeyWallet.create_new()
# print your private key so you can write it down for use later
print(wallet.address, wallet.private_key)

0xb67748C197cb7a42923dCc8039262F70E4d0Fe5D 0x4b7d1c2b24b2523aa74c71a6447fec8c74020e44b9fdcdb4804b71374a3bc785


In [146]:
from emp.orderly.onboarding import create_account, add_access_key

# create a new account on orderly
await create_account(wallet)

'0xbc0281ce183cc61bc7583d7685c1cc884e63fdd62fba92861ccbd4564781a383'

In [147]:
from emp.orderly.utils.ed25519 import publickey
from base58 import b58encode

# NOTE: you can make as many orderly keys as you want, but this one is generated from your
#       wallets private key.  Keep this PRIVATE!
key_bytes = publickey(bytes.fromhex(wallet.private_key.removeprefix("0x")))
orderly_key = "ed25519:%s" % b58encode(key_bytes).decode("utf-8")

In [148]:
# register your new orderly key as a signer for your wallet address
await add_access_key(wallet, orderly_key)

'ed25519:45L234JW5XeDjnwGFWg3mrhRAQuU66WAmpqwR93Vn1pN'

In [149]:
from emp.orderly.onboarding.faucet import request_testnet_funds

# get some testnet USDC from the faucet
await request_testnet_funds(wallet)

True

In [150]:
from emp.orderly import EmpyrealOrderlySDK
from emp.orderly.utils.orderly_id import from_address

# now you can initialize the SDK
sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=from_address(wallet.address), is_testnet=True)

In [151]:
from decimal import Decimal
from emp.orderly_types import PerpetualAssetType, MarketInfo, OrderResponse, PositionData
from pprint import pprint

bitcoin_asset_info = await sdk.asset_info(PerpetualAssetType.BTC)
pprint(bitcoin_asset_info)

{'data': {'base_imr': 0.02,
          'base_max': 20,
          'base_min': 1e-05,
          'base_mmr': 0.012,
          'base_tick': 1e-05,
          'cap_funding': 0.003,
          'cap_ir': 0.0004,
          'claim_insurance_fund_discount': 0.005,
          'created_time': 1693820915671,
          'deviation_factor': 10,
          'floor_funding': -0.003,
          'floor_ir': -0.0004,
          'funding_period': 8,
          'imr_factor': 4.35e-07,
          'interest_rate': 0.0001,
          'liquidation_tier': 1,
          'liquidator_fee': 0.006,
          'mark_index_price_deviation_cap': 1.03,
          'mark_index_price_deviation_floor': 0.97,
          'min_notional': 10,
          'price_range': 0.03,
          'price_scope': 0.4,
          'quote_max': 200000,
          'quote_min': 0,
          'quote_tick': 0.1,
          'std_liquidation_fee': 0.012,
          'symbol': 'PERP_BTC_USDC',
          'updated_time': 1724397608672},
 'success': True,
 'timestamp': 172754175

In [152]:
# this is the min tick size for an asset
base_min = Decimal(str(bitcoin_asset_info['data']['base_min']))
quote_tick = Decimal(str(bitcoin_asset_info['data']['quote_tick']))

print(f"base_min: {base_min}\nquote_tick: {quote_tick}")

base_min: 0.00001
quote_tick: 0.1


In [153]:
bitcoin_market_info: MarketInfo = await sdk.get_market_info(PerpetualAssetType.BTC)
print(bitcoin_market_info)

symbol=<PerpetualAssetType.BTC: 'PERP_BTC_USDC'> index_price=Decimal('65566.39') mark_price=Decimal('65575.3') sum_unitary_funding=Decimal('32330.9') est_funding_rate=Decimal('0.0001') last_funding_rate=Decimal('0.00010056') next_funding_time=datetime.datetime(2024, 9, 29, 0, 0, tzinfo=TzInfo(UTC)) open_interest=Decimal('304.7272') last_24h_open=Decimal('66198.7') last_24h_close=Decimal('65587.1') last_24h_high=Decimal('66311.3') last_24h_low=Decimal('65314.2') last_24h_volume=Decimal('148.30347') last_24h_amount=Decimal('9763044.91843')


In [154]:
# the market info has information about the asset, such as its index price, mark price, vunding rates, volume, and more
bitcoin_price = bitcoin_market_info.index_price
print(bitcoin_price)

65566.39


In [155]:
# we want to buy $25, so let's divide 25 by the bitcoin price.  We're using the decimal module to keep precision in our
# decimal numbers
purchase_amount = (Decimal('25') / bitcoin_price).quantize(base_min)
print(purchase_amount)

0.00038


In [156]:
# let's make the trade and open our position
while True:
    try:
        response: OrderResponse = await sdk.make_order( purchase_amount, PerpetualAssetType.BTC, is_buy=True)
    except ValueError:
        continue
    break
positions: PositionData = await sdk.positions()
print(response, "\n\n", positions)

order_id=1298304276 client_order_id=None order_type='MARKET' order_price=None order_quantity=Decimal('0.00038') order_amount=None 

 margin_ratio=Decimal('40.1376') initial_margin_ratio=Decimal('0.1') maintenance_margin_ratio=Decimal('0.012') open_margin_ratio=Decimal('40.1376') current_margin_ratio_with_orders=Decimal('40.1376') initial_margin_ratio_with_orders=Decimal('0.1') maintenance_margin_ratio_with_orders=Decimal('0.012') total_collateral_value=Decimal('999.981629') free_collateral=Decimal('997.4902502') rows=[Position(symbol=<PerpetualAssetType.BTC: 'PERP_BTC_USDC'>, position_qty=Decimal('0.00038'), cost_position=Decimal('24.932159'), last_sum_unitary_funding=Decimal('32330.9'), pending_long_qty=Decimal('0.0'), pending_short_qty=Decimal('0.0'), settle_price=Decimal('65610.94473684'), average_open_price=Decimal('65571.6'), unsettled_pnl=Decimal('-0.018371'), mark_price=Decimal('65562.6'), est_liq_price=Decimal('0.0'), timestamp=datetime.datetime(2024, 9, 28, 16, 43, 3, 553000, 

In [157]:
for position in positions.rows:
    print(position)

symbol=<PerpetualAssetType.BTC: 'PERP_BTC_USDC'> position_qty=Decimal('0.00038') cost_position=Decimal('24.932159') last_sum_unitary_funding=Decimal('32330.9') pending_long_qty=Decimal('0.0') pending_short_qty=Decimal('0.0') settle_price=Decimal('65610.94473684') average_open_price=Decimal('65571.6') unsettled_pnl=Decimal('-0.018371') mark_price=Decimal('65562.6') est_liq_price=Decimal('0.0') timestamp=datetime.datetime(2024, 9, 28, 16, 43, 3, 553000, tzinfo=TzInfo(UTC)) imr=Decimal('0.1') mmr=Decimal('0.012') IMR_withdraw_orders=Decimal('0.1') MMR_with_orders=Decimal('0.012') pnl_24_h=Decimal('0.0') fee_24_h=Decimal('0.014951')


In [158]:
# now we can see our current bitcoin position (assuming we have no other positions open)
print([position for position in positions.rows if position.symbol == PerpetualAssetType.BTC])

[Position(symbol=<PerpetualAssetType.BTC: 'PERP_BTC_USDC'>, position_qty=Decimal('0.00038'), cost_position=Decimal('24.932159'), last_sum_unitary_funding=Decimal('32330.9'), pending_long_qty=Decimal('0.0'), pending_short_qty=Decimal('0.0'), settle_price=Decimal('65610.94473684'), average_open_price=Decimal('65571.6'), unsettled_pnl=Decimal('-0.018371'), mark_price=Decimal('65562.6'), est_liq_price=Decimal('0.0'), timestamp=datetime.datetime(2024, 9, 28, 16, 43, 3, 553000, tzinfo=TzInfo(UTC)), imr=Decimal('0.1'), mmr=Decimal('0.012'), IMR_withdraw_orders=Decimal('0.1'), MMR_with_orders=Decimal('0.012'), pnl_24_h=Decimal('0.0'), fee_24_h=Decimal('0.014951'))]


In [159]:
take_profit_price: Decimal = (bitcoin_price * Decimal('1.1')).quantize(quote_tick)
while True:
    try:
        await sdk.make_limit_order('0.0005', price=take_profit_price, asset=PerpetualAssetType.BTC, is_buy=False)
    except ValueError:
        continue
    break
print(take_profit_price)

72123.0


In [160]:
from emp.orderly_types import Order

# get all unfulfilled orders to see your open bitcoin order
orders: list[Order] = await sdk.orders(status="INCOMPLETE")
print(orders)

[Order(symbol=<PerpetualAssetType.BTC: 'PERP_BTC_USDC'>, status='NEW', side='SELL', order_id=1298304277, user_id=140806, price=Decimal('72123'), type='LIMIT', quantity=Decimal('0.0005'), amount=None, visible=Decimal('0.0005'), executed=Decimal('0'), total_fee=Decimal('0'), fee_asset='USDC', client_order_id=None, created_time=datetime.datetime(2024, 9, 28, 16, 43, 4, 1000, tzinfo=TzInfo(UTC)), updated_time=datetime.datetime(2024, 9, 28, 16, 43, 4, 87000, tzinfo=TzInfo(UTC)), average_executed_price=None, reduce_only=None, order_tag='EMPYREAL', realized_pnl=Decimal('0'))]


In [161]:
# now, let's close our open limit order
await sdk.close_orders(PerpetualAssetType.BTC)

{'success': True,
 'data': {'status': 'CANCEL_ALL_SENT'},
 'timestamp': 1727541784601}

In [162]:
# and we can close our bitcoin position
await sdk.close_position(PerpetualAssetType.BTC)

OrderResponse(order_id=1298304278, client_order_id=None, order_type='MARKET', order_price=None, order_quantity=Decimal('0.00038'), order_amount=None)

In [163]:
from emp.orderly import Strategy, EmpOrderly, crossover

# define a trading strategy.  The only method necessary is the next method,
# which can access the current/historical prices and make buy/sell orders and limit orders.
class SmaCross(Strategy):
    n1: int = 10
    n2: int = 40

    @classmethod
    def update_lags(cls, n1, n2):
        cls.n1 = n1
        cls.n2 = n2

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy(size=0.5)
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell(size=0.5)

In [164]:
print(sdk.account_id)
print(sdk.pvt_hex)

0xbc0281ce183cc61bc7583d7685c1cc884e63fdd62fba92861ccbd4564781a383
0x4b7d1c2b24b2523aa74c71a6447fec8c74020e44b9fdcdb4804b71374a3bc785


In [165]:
# next we can create a new instance of EmpOrderly with an initial amount of cash and a commission
sdk = EmpOrderly(
    sdk=sdk,
    cash=1000,
    commission=.0001,
    exclusive_orders=True
)

# Then we can load the strategy to the SDK, and load historical data to use for backtesting
sdk.set_strategy(SmaCross)

EmpOrderly(asset=None, cash=1000.0, commission=0.0001, margin=1.0, sdk=EmpyrealOrderlySDK(account_id='0xbc0281ce183cc61bc7583d7685c1cc884e63fdd62fba92861ccbd4564781a383', pvt_hex='0x4b7d1c2b24b2523aa74c71a6447fec8c74020e44b9fdcdb4804b71374a3bc785', is_testnet=True), trade_on_close=False, hedging=False, exclusive_orders=True)

In [166]:
from emp.orderly_types import Interval

while True:
    try:
        await sdk.load_data(
            lookback=5,
            interval=Interval.five_minute,
            asset=PerpetualAssetType.APT,
        )
    except ValueError:
        continue
    break

CancelledError: 

In [142]:
# backtest
sdk.backtest()

AttributeError: Load data using .load_data()

In [167]:
# plot
sdk.plot(show_price_data=False)
plt.show()

AttributeError: 'EmpOrderly' object has no attribute '_results'